# Titanic Top 4% with Ensemble Modeling
1. Introduction
2. Load and Check data
    * 2.1 load data
    * 2.2 Outlier detection
    * 2.3 joining train and test set
    * 2.4 check for null and missing values
3. Feature analysis
    * 3.1 Numerical values
    * 3.2 Categorical values
4. Filling missing Values
    * 4.1 Age
5. Feature engineering
    * 5.1 Name / Title
    * 5.2 Family Size
    * 5.3 Cabin
    * 5.4 Ticket
6. Modeling
    * 6.1 Simple modeling
        * 6.1.1 Cross validate models
        * 6.1.2 Hyperparameter tunning for best models
        * 6.1.3 Plot learning curves
        * 6.1.4 Feature importance of the tree based classifiers
    * 6.2 Ensemble modeling
        * 6.2.1 Combining models
    * 6.3 Prediction
        * 6.3.1 Predict and Submit results

## 1. Introduction
This is my first kernel at Kaggle. I chose the Titanic competition which is a good way to introduce feature engineering and ensemble modeling. Firstly, I will display some feature analyses then i'll focus on the feature engineering. Last part concerns modeling and predictiong the survival on the Titanic using an voting procedure.   
This script follows three main parts:
* Feature analysis
* Feature engineering
* Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')

## 2. Load and Check data
### 2.1 Load data

In [2]:
# Load data
##### Load train and Test set

train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
IDtest = test["PassengerId"]

### 2.2 Outlier detection

In [3]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])

이상치는 예측에 극적인 영향을 미칠 수 있으므로(특히 회귀 문제의 경우), 나는 이상치를 관리하기로 선택했습니다.     
나는 Tukey 방법(Tukey JW. , 1977)을 사용하여 분포값(IQR)의 1사분위수와 3사분위수 사이로 구성된 사분위수 범위를 정의하는 이상치를 감지했습니다. 이상치는 (IQR +- outlier step) 외부에 특성값이 있는 행입니다.   
숫자 값 특성(Age, SibSp, Parch, Fare)에서 이상치를 감지하기로 결정했습니다. 그런 다음 이상치를 2개 이상의 이상값이 있는 행으로 간주했습니다.

In [4]:
train.loc[Outliers_to_drop] # Show the outliers rows

10개의 이상치를 감지했습니다. 28, 89와 342 승객은 티켓 요금이 높습니다.    
나머지 7개는 SibSp 값이 매우 높습니다.

In [5]:
# Drop outliers
train = train.drop(Outliers_to_drop, axis=0).reset_index(drop=True)

### 2.3 joining train and test set

In [6]:
## Join train and test datasets in order to obtain the same number of features during categorical conversion
train_len = len(train)
dataset = pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

범주형 변환 중에 동일한 수의 특성을 얻기 위해 훈련 세트와 테스트 세트를 합칩니다.(feature engineering 참조)

### 2.4 check for null and missing values

In [7]:
# Fill empty and NaNs values with NaN
dataset = dataset.fillna(np.nan)

# Check for Null values
dataset.isnull().sum()

Age와 Cabin 특성이 결측값의 중요한 부분을 차지합니다.   
생존 누락 값은 조인 테스트 데이터 세트에 해당합니다(Survived 열은 테스트 세트에 존재하지 않으며 훈련 세트와 데스트 세트를 연결할 때 NaN값으로 대체되었습니다.)

In [8]:
# Infos
train.info()
train.isnull().sum()

In [9]:
train.head()

In [10]:
train.dtypes

In [11]:
### Summarize data
# Summarie and statistics
train.describe()

### 3. Feature analysis
#### 3.1 Numerical values

In [12]:
# Correlation matrix between numerical values (SibSp Parch Age and Fare values) and Survived
g = sns.heatmap(train[['Survived', 'SibSp', 'Parch', 'Age', 'Fare']].corr(), annot=True, fmt='.2f', cmap='coolwarm')

Fare 특성만이 생존확률과 유의한 상관관계가 있는 것으로 보인다.    
다른 기능이 유용하지 않다는 의미는 아니다. 이러한 특성의 하위 개체군은 생존과 상관관계가 있을 수도 있다. 이를 확인하려면 이러한 특성을 자세히 조사해야 한다.

##### SibSp

In [13]:
# Explore SibSp feature vs Survived
g = sns.catplot(x='SibSp', y='Survived', data=train, kind='bar', height=6, palette='muted')
g.despine(left=True)
g = g.set_ylabels('Survival Probability')

형제/배우자가 많은 승객은 생존 가능성이 낮아 보인다.    
1인 승객(0 SibSp) 또는 2명(SibSp 1 or 2)은 생존할 확률이 더 높다.   
이 관찰은 매우 흥미롭다. 이러한 범주를 설명하는 새로운 특성을 고려할 수 있다.(feature engineering 참조)

#### Parch

In [14]:
# Explore Parch feature vs Survived
g = sns.catplot(x='Parch', y='Survived', data=train, kind='bar', height=6, palette='muted')
g.despine(left=True)
g = g.set_ylabels('Survival Probability')

소가족은 싱글(Parch 0), 중형(Parch 3,4) 및 대가족(Parch 5,6)보다 더 많은 생존 기회가 있다.   
부모/자녀가 3명인 승객은 생존에는 중요한 표준편차가 있으므로 주의하자!

#### Age

In [15]:
# Explore Age vs Survived
g = sns.FacetGrid(train, col='Survived')
g = g.map(sns.distplot, 'Age')

연령 분포는 꼬리 분포(아마도 가우스 분포)인 것 같다.   
우리는 연령 분포가 생존한 하위 집단과 생존하지 못한 하위 집단에서 동일하지 않다는 것을 알 수 있다. 실제로 살아남은 젊은 승객에 해당하는 피크가 있다. 우리는 또한 60-80세 사이의 승객들이 덜 살아남았다는 것을 알 수 있다.   
따라서 'Age'가 'Survived'와 상관관계가 없더라도 생존 가능성이 다소 있는 승객의 연령 범주가 있음을 알 수 있다.   
아주 어린 승객이 생존할 가능성이 더 높은 것 같다.

In [16]:
# Explore Age distribution
g = sns.kdeplot(train['Age'][(train['Survived'] == 0) & (train['Age'].notnull())], color='Red', shade=True)
g = sns.kdeplot(train['Age'][(train['Survived'] == 1) & (train['Age'].notnull())], ax=g, color='Blue', shade=True)
g.set_xlabel('Age')
g.set_ylabel('Frequency')
g = g.legend(['Not Survived', 'Survived'])

두 밀도를 중첩하면 아기와 아주 어린 아이들에게 해당하는 피크(0과 5사이)가 분명히 보인다.

#### Fare

In [17]:
dataset['Fare'].isnull().sum()

In [18]:
# Fill Fare missing values with the median value
dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())

하나의 누락된 값이 있으므로 예측에 중요한 영향을 미치지 않는 중앙값으로 채우기로 결정했다.

In [19]:
# Explore Fare distribution
g = sns.distplot(dataset['Fare'], color='m', label='Skewness : %.2f'%(dataset['Fare'].skew()))
g = g.legend(loc='best')

보시다시피 Fare 분포는 매우 편향되어 있다. 이로 인해 크기가 조정된 경우에도 모델에서 매우 높은 값의 과중화가 발생할 수 있다.    
이 경우 이러한 치우침을 줄이기 위해 로그 함수로 변환하는 것이 좋다.

In [20]:
# Apply log to Fare to reduce skewness distribution
dataset['Fare'] = dataset['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [21]:
g = sns.distplot(dataset['Fare'], color='b', label='Skewness : %.2f'%(dataset['Fare'].skew()))
g = g.legend(loc='best')

로그변환 후 치우침이 확연하게 감소했다.

### 3.2 Categorical values
#### Sex

In [22]:
# Explore Sex vs Survived
g = sns.barplot(x='Sex', y='Survived', data=train)
g = g.set_ylabel('Survival Probability')

In [23]:
train[['Sex', 'Survived']].groupby('Sex').mean()

남성이 여성보다 생존 확률이 낮다는 것은 명백하다.   
따라서 Sex는 생존 예측에 중요한 역할을 할 수 있다.   
타이타닉 영화(1997)를 본 사람들은 대피 중 이 문장을 기억할 것이라고 확신한다. "Women and children first"

#### Pclass

In [24]:
# Explore Pclass vs Survived
g = sns.catplot(x='Pclass', y='Survived', data=train, kind='bar', height=6, palette='muted')
g.despine(left=True)
g = g.set_ylabels('Survival Probability')

In [25]:
# Explore Pclass vs Survived by Sex
g = sns.catplot(x='Pclass', y='Survived', hue='Sex', data=train, kind='bar', height=6, palette='muted')
g.despine(left=True)
g = g.set_axis_labels('Pclass','Survival Probability')

승객 생존은 3가지 클래스에서는 동일하지 않다. 1등석 승객은 2등석 및 3등석 승객보다 생존 가능성이 더 높다.   
이 경향을 남성과 여성 승객만 볼 때도 보존된다.

#### Embarked

In [26]:
dataset['Embarked'].isnull().sum()

In [27]:
# Fill Embarked nan values of dataset with 'S' most frequent value
dataset['Embarked'] = dataset['Embarked'].fillna('S')

2개의 결측값이 존재하므로, 가장 빈도수가 높은 'Embarked' 값인 'S'로 null값을 채워주었다.

In [28]:
# Explore Embarked vs Survived
g = sns.catplot(x='Embarked', y='Survived', data=train, height=6, kind='bar', palette='muted')
g.despine(left=True)
g = g.set_ylabels('Survival Probability')

Cherbourg(C)에서 온 승객은 생존 가능성이 더 높아 보인다.  
여기서 세울 수 있는 가설은 Cherbourg(C)에서 온 승객Queenstown(Q), Southampton(S)에서 온 승객의 1등석 승객 비율이 더 높다는 것이다.   
Pclass vs Embarked 분포를 보자!

In [29]:
# Explore Pclass vs Embarked
g = sns.catplot('Pclass', col='Embarked', data=train, height=6, kind='count', palette='muted')
g.despine(left=True)
g = g.set_ylabels('Count')

실제로 3등석은 Southampton(S)와 Queenstown(Q)에서 온 승객에게 가장 빈번한 반면 Cherbourg에서 온 승객은 대부분 생존율이 가장 높은 1등석이다.  
이 시점에서는 1등석의 생존율이 높은 이유를 설명할 수 없다. 세울 수 있는 가설은 1등석 승객이 대피 과정에서 우선순위가 높았다는 것이다.

## 4. Filling missing Values
### 4.1 Age
* 보시다시피 Age 열에는 전체 데이터 세트에서 256개의 누락된 값이 있다.    
* 생존 가능성이 더 높은 하위 모집단(어린이)이 있으므로 연령 특성을 유지하고 결측값을 대치하는 것이 좋겠다.   
* 이 문제를 해결하기 위해 연령과 가장 상관관계가 높은 특성(Sex, Parch, Pclass, SibSp)을 살펴보자.

In [30]:
# Explore Age vs Sex, Parch, Pclass and SibSp
g = sns.catplot(y='Age', x='Sex', data=dataset, kind='box')
g = sns.catplot(y='Age', x='Sex', hue='Pclass', data=dataset, kind='box')
g = sns.catplot(y='Age', x='Parch', data=dataset, kind='box')
g = sns.catplot(y='Age', x='SibSp', data=dataset, kind='box')

연령 분포는 남성과 여성 하위 모집단에서 동일한 것으로 보이므로 성별은 연령을 예측하는 데 유용하지 않다.   
그러나 1등석 승객은 2등석 승객보다 나이가 많고, 2등석은 3등석 승객보다 나이가 많다.   
또한, 승객이 나이가 많을수록 부모/자녀가 많고 더 어린 형제/배우자가 있다.

In [31]:
# convert Sex into categorical value 0 for male and 1 for female
dataset['Sex'] = dataset['Sex'].map({'male': 0, 'female': 1})

In [32]:
g = sns.heatmap(dataset[['Age','Sex','SibSp','Parch','Pclass']].corr(), cmap='BrBG', annot=True)

상관관계 그래프는 Parch를 제외한 factorplot 관측값을 확인한다. Age는 Sex와 상관관계가 없지만, Pclass, Parch, SibSp와 음의 상관관계가 있다.    
Parch의 특성에 따른 연령 플롯에서 연령은 부모/자녀의 수에 따라 증가한다. 그러나 일반적인 상관관계는 음수이다.   
그래서 누락된 연령을 추정하기 위해 SibSp, Parch, Pclass를 사용하기로 결정했다.   
전략은 Pclass, Parch, SibSp에 따라 유사한 행의 중앙값 연령으로 Age를 채우는 것이다.

In [33]:
# Filling missing value of Age

# Fill Age with the median age of similar rows according to Pclass, Parch and SibSp
# Index of NaN age rows
index_NaN_age = list(dataset['Age'][dataset['Age'].isnull()].index)

for i in index_NaN_age:
    age_med = dataset['Age'].median()
    age_pred = dataset['Age'][((dataset['SibSp'] == dataset.iloc[i]['SibSp'])
                               &(dataset['Parch'] == dataset.iloc[i]['Parch'])
                              &(dataset['Pclass'] == dataset.iloc[i]['Pclass']))].median()
    if not np.isnan(age_pred):
        dataset['Age'].iloc[i] = age_pred
    else:
        dataset['Age'].iloc[i] = age_med

In [34]:
g = sns.catplot(x='Survived', y='Age', data=train, kind='box')
g = sns.catplot(x='Survived', y='Age', data=train, kind='violin')

생존한 하위 집단과 생존하지 않은 하위 집단에서 연령의 중앙값 사이에는 차이가 없다.   
그러나 생존한 승객의 violinplot에서 우리는 여전히 아주 어린 승객의 생존율이 더 높다는 것을 알 수 있다.

## 5. Feature engineering
### 5.1 Name/Title

In [35]:
dataset['Name'].head()

Name 특성에는 승객의 직위 정보가 포함되어있다.   
대피하는 동안 고유한 직위를 가진 일부 승객이 선호될 수 있으므로 모델에 추가하는 것이 좋을 것 같다.

In [36]:
# Get Title from Name
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset['Name']]
dataset['Title'] = pd.Series(dataset_title)
dataset['Title'].head()

In [37]:
g = sns.countplot(x='Title', data=dataset)
g = plt.setp(g.get_xticklabels(), rotation=45)

데이터 세트에는 17개의 title이 있으며 대부분은 매우 드물며 4개의 범주로 그룹화할 수 있다.

In [38]:
# Convert to categorical values Title
dataset['Title'] = dataset['Title'].replace(['Lady','the Countess','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
dataset['Title'] = dataset['Title'].map({'Master':0, 'Miss':1, 'Ms':1, 'Mme':1, 'Mlle':1, 'Mrs':1, 'Mr':2,'Rare':3})
dataset['Title'] = dataset['Title'].astype(int)

In [39]:
g = sns.countplot(dataset['Title'])
g = g.set_xticklabels(['Master','Miss/Ms/Mme/Mlle/Mrs','Mr','Rare'])

In [40]:
g = sns.catplot(x='Title', y='Survived', data=dataset, kind='bar')
g = g.set_xticklabels(['Master','Miss-Mrs','Mr','Rare'])
g = g.set_ylabels('Survival Probability')

"Women and children first"   
희귀 title을 가진 승객이 생존할 확률이 더 높다는 점은 흥미롭다.

In [41]:
# Drop Name variable
dataset.drop(labels=['Name'], axis=1, inplace=True)

### 5.2 Family size
대가족은 대피하는 동안 그들의 자매/형제/부모를 찾느라 더 많은 어려움을 겪을 것이라고 상상할 수 있다. 그래서 SibSp, Parch 및 1(승객 포함)의 합계인 'Fize'(family size) 특성을 만들어보자.

In [42]:
# Create a family size descriptor from SibSp and Parch
dataset['Fsize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [43]:
g = sns.catplot(x='Fsize', y='Survived', data=dataset, kind='point')
g = g.set_ylabels('Survival Probability')

가족 규모가 중요한 역할을 하는 것으로 보이며, 대가족의 경우 생존 확률이 가장 낮다.   
또한 4가지 범주의 가족 규모를 만들기로 결정했다.

In [44]:
# Create new feature of family size
dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if s == 2 else 0)
dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['Fsize'].map(lambda s: 1 if s >= 5 else 0)

In [45]:
g = sns.catplot(x='Single', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')
g = sns.catplot(x='SmallF', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')
g = sns.catplot(x='MedF', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')
g = sns.catplot(x='LargeF', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')

가족 규모 범주의 catplot은 중소형 가족이 1인 승객 및 대가족보다 생존할 기회가 더 많다는 것을 보여준다.

In [46]:
# convert to indicator values Title and Embarked
dataset = pd.get_dummies(dataset, columns=['Title'])
dataset = pd.get_dummies(dataset, columns=['Embarked'], prefix='Em')

In [47]:
dataset.head()

이 단계에서, 우리는 22개의 특성을 가지고 있다.

### 5.3 Cabin

In [48]:
dataset['Cabin'].head()

In [49]:
dataset['Cabin'].describe()

In [50]:
dataset['Cabin'].isnull().sum()

Cabin 특성 열에는 292개의 값과 1007개의 결측값이 있다.   
객실이 없는 승객은 객실 번호 대신 누락된 값이 표시된다고 가정하자.

In [51]:
dataset['Cabin'][dataset['Cabin'].notnull()].head()

In [52]:
# Replace the Cabin number by the type of Cabin 'X' if not
dataset['Cabin'] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin']])

객실의 첫 글자는 데스크를 나타내며 타이타닉에서 승객의 가능한 위치를 나타내기 때문에 이 정보만 유지하기로 했다.

In [53]:
g = sns.countplot(dataset['Cabin'], order=['A','B','C','D','E','F','G','T','X'])

In [54]:
g = sns.catplot(y='Survived', x='Cabin', data=dataset, kind='bar',
                order=['A','B','C','D','E','F','G','T','X'])
g = g.set_ylabels('Survival Probability')

객실이 있는 승객의 수가 적기 때문에 생존 확률은 중요한 표준편차를 가지며 다른 데스크에 있는 승객의 생존 확률을 구별할 수 없다.   
그러나 객실이 있는 승객은 일반적으로 (X)가 없는 승객보다 생존 가능성이 더 높다는 것을 알 수 있다.    
객실 B, C, D, E, F의 경우 특히 그렇다.

In [55]:
dataset = pd.get_dummies(dataset, columns=['Cabin'], prefix='Cabin')

### 5.4 Ticket

In [56]:
dataset['Ticket'].head()

동일한 prefix를 공유하는 티켓이 동일한 객실에 대해 예약될 수 있다는 것을 의미할 수 있다. 따라서 선박 내 객실의 실제 배치로 이어질 수 있다.    
prefix가 같은 티켓은 클래스와 생존이 비슷할 수 있다.    
그래서 티켓 특성 열을 티켓 prefix로 바꾸기로 결정했다.더 많은 정보를 얻을 수 있을 것이다. 

In [57]:
# Treat Ticket by extracting the ticket prefix. When there is no prefix, it returns X.

Ticket = []
for i in list(dataset.Ticket):
    if not i.isdigit():
        Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) # Take prefix
    else:
        Ticket.append('X')
        
dataset['Ticket'] = Ticket
dataset['Ticket'].head()

In [58]:
dataset = pd.get_dummies(dataset, columns = ['Ticket'], prefix='T')

In [59]:
# Create categorical value for Pclass
dataset['Pclass'] = dataset['Pclass'].astype('category')
dataset = pd.get_dummies(dataset, columns = ['Pclass'], prefix='Pc')

In [60]:
# Drop useless variables
dataset.drop(labels = ['PassengerId'], axis=1, inplace=True)

In [61]:
dataset.head()

## 6. Modeling

In [62]:
## Seperate train dataset and test dataset

train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=['Survived'], axis=1, inplace=True)

In [63]:
## Seperate train features and label

train['Survived'] = train['Survived'].astype(int)
Y_train = train['Survived']
X_train = train.drop(labels = ['Survived'], axis=1)

### 6.1 Simple modeling
#### 6.1.1 Cross validate models
10개의 인기 있는 분류기를 비교하고 계층화된 kfold 교차 검증 절차를 통해 각 분류기의 평균 정확도를 평가했다.   
* SVC
* Decision Tree
* AdaBoost
* Random Forest
* Extra Trees
* Gradient Boosting
* Multiple layer perceptron (neural network)
* KNN
* Logistic regression
* Linear Discriminant Analysis

In [64]:
# Cross validate model with kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [65]:
# Modeling step Test differents algorithms
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state), learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state=random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier, X_train, y=Y_train, scoring='accuracy', cv=kfold, n_jobs=4))
    
cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
    
cv_res = pd.DataFrame({'CrossValMeans':cv_means, 'CrossValerrors':cv_std,
                       'Algorithm':['SVC','DecisionTree','AdaBoost','RandomForest',
                                   'ExtraTrees','GradientBoosting','MultipleLayerPerceptron',
                                   'KNeighbors','LogisticRegression','LinearDiscriminantAnalysis']})

g = sns.barplot('CrossValMeans','Algorithm', data=cv_res, palette='Set3', orient='h', **{'xerr':cv_std})
g.set_xlabel('Mean Accuracy')
g = g.set_title('Cross validation scores')

앙상블 모델링을 위해 SVC, AdaBoost, RandomForest, ExtraTrees, GradientBoosting 분류기를 선택하기로 결정했다.

#### 6.1.2 Hyperparameter tunning for best models
AdaBoost, ExtraTrees, RandomForest, GradientBoosting, SVC 분류기에 대한 그리드 검색 최적화를 수행했다.   
cpu가 4개이므로 'n_jobs' 매개변수를 4로 설정했다. 계산 시간이 확실히 줄어든다.   
하지만 주의!! 이 단계는 시간이 오래 걸릴 수 있다. 이분은 4개의 cpu에서 총 15분이 걸렸다고 한다.

In [66]:
# META MODELING WITH ADABOOST, RF, EXTRATREES and GRADIENTBOOSTING

# AdaBoost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {'base_estimator__criterion':['gini','entropy'],
                 'base_estimator__splitter':['best','random'],
                 'algorithm':['SAMME','SAMME.R'],
                 'n_estimators':[1,2],
                 'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 1.5]}

gsadaDTC = GridSearchCV(adaDTC, param_grid=ada_param_grid, cv=kfold, scoring='accuracy', n_jobs=4, verbose=1)

gsadaDTC.fit(X_train,Y_train)

ada_best = gsadaDTC.best_estimator_

In [67]:
gsadaDTC.best_score_

In [68]:
# ExtraTrees
ExtC = ExtraTreesClassifier()

## Search grid for optimal parameters
ex_param_grid = {'max_depth':[None],
                'max_features':[1,3,10],
                'min_samples_split':[2,3,10],
                'min_samples_leaf':[1,3,10],
                'bootstrap':[False],
                'n_estimators':[100,300],
                'criterion':['gini']}

gsExtC = GridSearchCV(ExtC, param_grid=ex_param_grid,
                     cv=kfold, scoring='accuracy', n_jobs=4, verbose=1)

gsExtC.fit(X_train, Y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [69]:
# RFC Parameters tunning
RFC = RandomForestClassifier()

## Search grid for optimal parameters
rf_param_grid = {'max_depth':[None],
                'max_features':[1,3,10],
                'min_samples_split':[2,3,10],
                'min_samples_leaf':[1,3,10],
                'bootstrap':[False],
                'n_estimators':[100,300],
                'criterion':['gini']}

gsRFC = GridSearchCV(RFC, param_grid=rf_param_grid,
                    cv=kfold, scoring='accuracy', n_jobs=4, verbose=1)

gsRFC.fit(X_train, Y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [70]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss':['deviance'],
                'n_estimators':[100,200,300],
                'learning_rate':[0.1, 0.05, 0.01],
                'max_depth':[4,8],
                'min_samples_leaf':[100,150],
                'max_features':[0.3,0.1]}

gsGBC = GridSearchCV(GBC, param_grid=gb_param_grid,
                    cv=kfold, scoring='accuracy', n_jobs=4, verbose=1)

gsGBC.fit(X_train, Y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

In [71]:
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel':['rbf'],
                 'gamma':[0.001, 0.01, 0.1, 1],
                 'C':[1,10,50,100,200,300,1000]}

gsSVMC = GridSearchCV(SVMC, param_grid=svc_param_grid,
                     cv=kfold, scoring='accuracy', n_jobs=4, verbose=1)

gsSVMC.fit(X_train, Y_train)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

#### 6.1.3 Plot learning curves
학습 곡선은 훈련 세트에 대한 과적합 효과와 정확도에 대한 훈련 크기의 영향을 확인하는 좋은 방법이다.

In [72]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                       n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    '''
    Generate a simple plot of the test and training learning curve
    '''
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    train_sizes, train_scores, test_scores=learning_curve(
    estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                    train_scores_mean + train_scores_std, alpha=0.1, color='r')
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                    test_scores_mean + test_scores_std, alpha=0.1, color='g')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score')
    plt.legend(loc='best')
    return plt

g = plot_learning_curve(gsRFC.best_estimator_, 'RF learning curves',
                       X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsExtC.best_estimator_,'ExtraTrees learning curves',
                       X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsSVMC.best_estimator_, 'SVC learning curves',
                       X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsadaDTC.best_estimator_, 'AdaBoost learning curves',
                       X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsGBC.best_estimator_, 'GradientBoosting learning curves',
                       X_train, Y_train, cv=kfold)

GradientBoosting, AdaBoost 분류기는 훈련 세트를 과적합하는 경향이 있다. 증가하는 교차 검증 곡선에 따르면 GradientBoosting과 AdaBoost는 더 많은 훈련 예제에서 더 나은 성능을 보일 수 있다.    
SVC, ExtraTrees 분류기는 훈련 및 교차 검증 곡선이 서로 가깝기 때문에 예측을 더 잘 일반화하는 것 같다.

#### 6.1.4 Feature importance of tree based classifiers
승객 생존 예측을 위한 가장 유익한 특성을 보기 위해 4개의 트리 기반 분류기에 대한 특성 중요도를 표시했다.

In [73]:
nrows = ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='all', figsize=(15,15))

names_classifiers = [('AdaBoosting', ada_best), ('ExtraTrees', ExtC_best),
                    ('RandomForest', RFC_best), ('GradientBoosting', GBC_best)]

nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        indices = np.argsort(classifier.feature_importances_)[::-1][:40]
        g = sns.barplot(y=X_train.columns[indices][:40],
                       x=classifier.feature_importances_[indices][:40],
                       orient='h', ax=axes[row][col])
        g.set_xlabel('Relative importance', fontsize=12)
        g.set_ylabel('Features', fontsize=12)
        g.tick_params(labelsize=9)
        g.set_title(name + ' feature importance')
        nclassifier += 1

4가지 트리 기반 분류기(AdaBoost, ExtraTrees, RandomForest, GradientBoosting)에 대한 특성 중요도를 표시한다.    
4개의 분류기는 상대적으로 중요도에 따라 서로 다른 상위 특성을 가진다. 이는 그들의 예측이 동일한 특성을 기반으로 하지 않는다는 것을 의미한다. 그럼엗 불구하고 Fare, Title_2, Age, Sex와 같이 분류에 대한 몇가지 공통적인 중요한 특성을 공유한다.   
Mrs/Mlle/Mme/Miss/Ms 카테고리를 나타내는 Title_2는 Sex와 높은 상관관계를 보인다.   
우리는 다음과 같이 말할 수 있다.:    
* Pc_1, Pc_2, Pc_3, Fare는 승객의 일반적인 사회적 지위를 나타낸다.
* Sex, Title_2(Mrs/Mlle/Mme/Miss/Ms), Title_3(Mr)은 성별을 나타낸다.
* Age, Title_1(Master)은 승객의 나이를 나타낸다.
* Fsize, LargeF, MedF, Single은 승객 가족의 크기를 나타낸다.   

이 4가지 분류기의 특성 중요도에 따르면, 생존 예측은 배 안에서의 위치보다 승객의 연령, 성별, 가족 규묘 및 사회적 지위와 더 관련이 있는 것으로 보인다.

In [74]:
test_Survived_RFC = pd.Series(RFC_best.predict(test), name='RFC')
test_Survived_ExtC = pd.Series(ExtC_best.predict(test), name='ExtC')
test_Survived_SVMC = pd.Series(SVMC_best.predict(test), name='SVC')
test_Survived_AdaC = pd.Series(ada_best.predict(test), name='Ada')
test_Survived_GBC = pd.Series(GBC_best.predict(test), name='GBC')

# Concatenate all classifier results
ensemble_results = pd.concat([test_Survived_RFC, test_Survived_ExtC,
                             test_Survived_AdaC, test_Survived_GBC,
                             test_Survived_SVMC], axis=1)

g = sns.heatmap(ensemble_results.corr(), annot=True)

예측은 AdaBoost가 다른 분류기와 비교할 때를 제외하고 5개의 분류기에 대해 매우 유사한 것으로 보인다.    
5개의 분류기는 거의 동일한 예측을 제공하지만 몇 가지 차이점이 있다. 5개의 분류기 예측 간의 이러한 차이는 앙상블 투표를 고려하기에 충분하다.

### 6.2 Ensemble modeling
#### 6.2.1 Combining models
5개의 분류기에서 나오는 예측을 결합하기 위해 투표 분류기를 선택했다.   
각 투표의 확률을 고려하기 위해 'soft'인자를 투표 매개변수에 전달하였다.

In [75]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
                                      ('svc', SVMC_best), ('adac', ada_best),
                                      ('gbc', GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train, Y_train)

### 6.3 Prediction
#### 6.3.1 Predict and Submit results

In [76]:
test_Survived = pd.Series(votingC.predict(test), name='Survived')

results = pd.concat([IDtest, test_Survived], axis=1)

results.to_csv('ensemble_python_voting.csv', index=False)